In [1]:
import os
from core.project import Project
from utils.config import DEFAULT_LITHO_LEXICON, DEFAULT_LITHO_LEGEND

In [2]:
from core.orm import Base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from utils.visual import legend_from_attributes # une legende par défaut devrait être charge si aucune n'est passée issue #113

In [3]:
pollutants = ['Chlorure', 'PCB_28', 'PCB_138', 'PCB_118', 'BTEX_tot', 'S_tot', 'PCB_tot', 'PCB_180', 'PCB_101',
              'PCB_153', 'PCB_52', 'MTBE', 'Pyr', 'Bnz', 'Alp_C6-C8', 'Toln', 'Pb', 'Bnz(ghi)peryl', 'CN_cplx',
              'Cr_VI', 'Cis-1.2-DCEn', 'Ar_C6-C7', 'Cd', 'Phenol', '1.2-DCE', 'CN_libre', 'Bnz(a)pyr', 'Cu', 
              'Bnz(k)flranth', 'Anthrc', 'C16-C21', '1.1-DCEn', 'Chloroforme', 'Styr', 'Ni', 'EOX', 'Naphta', 
              'DCM', '1.1-DCE', 'C5-C8', 'As', '1.1.2-TCE', 'Bnz(b)flranth', 'Cr', 'HC_tot_C10-C40', 'HAP_tot_EPA', 
              'Co', 'Phenanthr', 'CN_tot', 'C12-C16', 'TCEyn', '(cis.trans)-1.2-DCEn_tot', 'Alp_C5-C6', 'Chrys', 
              'Trans-1.2-DCEyl', '1.2-DCP', 'Ar_C7-C8', '1.1.1-TCE', 'Zn', 'Indeno(1.2.3-cd)pyr', 'Alp_C8-C10', 
              'Flranth', 'HC_tot_C10-C35', 'Acenaphtyl', 'C8-C10', 'Bnz(a)anthrc', 'Dibnz(ah)anthrc', 'Hg', 
              'Ar_C8-C10', 'C10-C12', 'C35-C40', 'P-M-Xyl', 'CVinyl', 'Acenaphtn', 'TetraCEyn', 'Xyl', 'C21-C35', 
              'TCM', 'O-Xyl']
legend_dict = legend_from_attributes([('lithology', DEFAULT_LITHO_LEGEND)] + pollutants)

In [4]:
db_name = './new_db.db'
if os.path.exists(db_name):
    os.remove(db_name)

In [5]:
# This should become a constructor of Project (use polymorphism)
def load(db_name, **kwargs):
    """ loads a project from a project database"""
    legend_dict = kwargs.pop('legend_dict', legend_from_attributes([('lithology', DEFAULT_LITHO_LEGEND)]))
    litho_lexicon = kwargs.pop('litho_lexicon', None)
    project_name = kwargs.pop('project_name', db_name.rstrip('.db'))
    verbose = kwargs.pop('verbose', False)
    
    engine = create_engine(f"sqlite:///{db_name}", echo=verbose)
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    session = Session()

    p = Project(session, name=project_name, legend_dict=legend_dict, lexicon=litho_lexicon)
    p.refresh()
    session.close()
    p.update_legend_cmap(compute_all_attrib=True, verbose=False)
    return p

## Create a new project from an empty database

In [6]:
p = load(db_name, legend_dict=legend_dict)
#p.plot_3d( window_size=(800, 600))

/home/su530201/PycharmProjects/GSDMA/core/project.py:75: SAWarning: relationship 'ComponentOrm.intervals' will copy column Components.id to column Linkintervalcomponent.comp_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Components.id to Linkintervalcomponent.comp_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="components"' to the 'ComponentOrm.intervals' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
  self.boreholes_orm = self.session.query(BoreholeOrm).all()
/home/su530201/PycharmProjects/GSDMA/core/project.py:75: SAWarning: relationship 'ComponentOrm.intervals' will c

In [7]:
from core.orm import BoreholeOrm, IntervalOrm, PositionOrm, ComponentOrm

In [8]:
# This function should be converted into a method of Project
def insert_borehole(project, bh_dict, verbose=True):
    bh_orm = BoreholeOrm(id=bh_dict['id'], date=bh_dict.pop('date', None), length=bh_dict.pop('length', None), diameter=bh_dict.pop('diameter', None))
    intv_id = project.find_next_id(IntervalOrm)
    bh_orm.intervals_values.update({intv_id: {'description' : "{'lithology': 'sand'}",
                                'interval_number': 0,
                                'top': PositionOrm(**bh_dict['top']),
                                'base': PositionOrm(**bh_dict['base'])}})
    # TODO associate with a special 'borehole' component
    new_component = ComponentOrm(description="{'borehole': 'piezometer'}")
    project.session.add(new_component)
    project.commit()
    project.add_borehole(bh_orm, verbose)

In [9]:
insert_borehole(p, {'id':'F01', 'length': 12.1, 'diameter':0.5, 'top':{'middle': 0., 'x': 100., 'y': 200.}, 'base':{'middle': -10., 'x': 100., 'y': 200.}})

Boreholes in the project :  0
Boreholes in the project :  0
description <core.orm.IntervalOrm>(Id=0, top=<core.orm.PositionOrm object at 0x7fdaef711220>, base=<core.orm.PositionOrm object at 0x7fdaef6079a0>, Description={'lithology': 'sand'}, Components=[])
depth: [-10.0]
Borehole geometry created successfully !

F01  added


/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/pyvista/core/dataset.py:1332: PyvistaDeprecationWarning: Use of `cell_arrays` is deprecated. Use `cell_data` instead.
  warnings.warn(


In [10]:
p.boreholes_orm

[<core.orm.BoreholeOrm>(Name=F01, Length=12.1, Diameter=0.5, Intervals=1)]

In [11]:
p.get_component_id_from_description("{'borehole': 'piezometer'}")

1

In [12]:
p.plot_3d( window_size=(800, 600), str_annotations=False)

In [15]:
p.boreholes_orm[0].diameter

0.5